## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler as SS
import pandas as pd
import tensorflow as tf
from tensorflow import keras as k

#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("Resources/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
{column: len(df[column].unique()) for column in df.columns}

{'APPLICATION_TYPE': 17,
 'AFFILIATION': 6,
 'CLASSIFICATION': 71,
 'USE_CASE': 5,
 'ORGANIZATION': 4,
 'STATUS': 2,
 'INCOME_AMT': 9,
 'SPECIAL_CONSIDERATIONS': 2,
 'ASK_AMT': 8747,
 'IS_SUCCESSFUL': 2}

In [4]:

df['ASK_AMT'].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [5]:
df['ASK_AMT'] = df['ASK_AMT'].apply(lambda x: 5000 if x > 5000 else 'other')
df['ASK_AMT'].value_counts()

other    25398
5000      8901
Name: ASK_AMT, dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace

df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(lambda x: x if x in ['T3', 'T4', 'T6', 'T5', 'T19', 'T8', 'T7'] else 'other')
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
df['CLASSIFICATION'].value_counts().head(20)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Name: CLASSIFICATION, dtype: int64

In [9]:
df['CLASSIFICATION'] = df['CLASSIFICATION'].apply(lambda x: x if x in ['C1000', 'C2000', 'C1200', 'C3000', 'C2100', 'C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700'] else 'other')
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [10]:
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,other,Independent,C1000,ProductDev,Association,1,0,N,other,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,5000,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,other,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,5000,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,5000,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,other,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,other,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,other,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,other,1


In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dfr = pd.get_dummies(df)

In [12]:
# Split our preprocessed data into our features and target arrays
X = dfr.drop('IS_SUCCESSFUL', axis=1)
y = dfr['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
_X, X_, _y, y_ = tts(X, y)

In [13]:
# Create a StandardScaler instances
scaler = SS()

# Fit the StandardScaler
scaler.fit(_X)

# Scale the data
_Xs = scaler.transform(_X)
Xs_ = scaler.transform(X_)

In [16]:
_Xs.shape

(25724, 49)

## Compile, Train and Evaluate the Model

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = k.models.Sequential()

# First hidden layer
nn.add(k.layers.Dense(activation='relu', units=150, input_dim=49))

# Second hidden layer
nn.add(k.layers.Dense(activation='relu', units=50))

# Output layer
nn.add(k.layers.Dense(activation='sigmoid', units=1))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 150)               7500      
                                                                 
 dense_8 (Dense)             (None, 50)                7550      
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 15,101
Trainable params: 15,101
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(optimizer='adam', metrics='accuracy', loss='binary_crossentropy')

In [27]:
# Train the model
fit = nn.fit(_Xs, _y, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5558 - accuracy: 0.7271
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5525 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5503 - accuracy: 0.7311
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5492 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5481 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5469 - accuracy: 0.7324
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5462 - accuracy: 0.7321
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5460 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5447 - accuracy: 0.7336
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5439 - accura

804/804 [==============================] - 3s 4ms/step - loss: 0.5317 - accuracy: 0.7398
Epoch 82/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5318 - accuracy: 0.7397
Epoch 83/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5313 - accuracy: 0.7401
Epoch 84/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5317 - accuracy: 0.7407
Epoch 85/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5313 - accuracy: 0.7397
Epoch 86/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7400
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5314 - accuracy: 0.7403
Epoch 88/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5312 - accuracy: 0.7395
Epoch 89/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5315 - accuracy: 0.7393
Epoch 90/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5309 - accuracy: 

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(Xs_, y_,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5588 - accuracy: 0.7350 - 1s/epoch - 5ms/step
Loss: 0.5587630867958069, Accuracy: 0.7350437045097351


In [ ]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE